In [11]:
import numpy as np
import pandas as pd
import pandas_datareader
import yaml
with open('./alpha_key.yml', 'r') as f:
    ALPHA_API_KEY = yaml.safe_load(f)['alpha_api_key']
    

In [12]:
tickers = ['AAPL']
lookback=200
metric = 'low'
pc_metric = f'{metric}_percent_change'
target_name = f'{pc_metric}_target'
def format_dates(daily_stocks_data):
    df = daily_stocks_data.copy() 
    df['date']=df.index
    df.reset_index(inplace=True, drop=True)
    return df
def add_percent_change(daily_stocks_data, metric):
    percents = list()
    for index, row in daily_stocks_data.iterrows():
        old = row[metric]
        try:
            new = daily_stocks_data.iloc[index + 1][metric]
        except Exception as e:
            percents.append(np.nan) ## no next value, so this is undefined
            continue
        percents.append((new-old)/new)
    cp_df = daily_stocks_data.copy()
    cp_df[f'{metric}_percent_change']=percents
    return cp_df
def to_ts_df(daily_stocks_data, lookback, metric):
    ## column names
    columns = list()
    for i in range(lookback):
        columns.append(f'{metric}_{i}')
    columns.append(f'{metric}_target')
    df = pd.DataFrame(columns=columns)
    ## columns
    data = daily_stocks_data[metric].to_numpy()
    for index, col in enumerate(df.columns):
        df[col] = data[index:len(data)-lookback+index]
    ## dates index
    dates = daily_stocks_data.date.to_numpy()[:-lookback]
    df.insert(0, 'dates', dates)
    return df

In [ ]:
daily_stocks_data_raw = pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=tickers, api_key=ALPHA_API_KEY, function='TIME_SERIES_DAILY').read()
daily_stocks_data = format_dates(daily_stocks_data_raw) 
daily_stocks_data = add_percent_change(daily_stocks_data, metric)
display(daily_stocks_data)
sns.lineplot(data=daily_stocks_data[pc_metric])
plt.show()
ts_df = to_ts_df(daily_stocks_data, lookback, pc_metric)
display(ts_df)
# sns.pairplot(daily_stocks_data)
# plt.show()